This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY =

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
sr = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?collapse=none&start_date=2017-10-01&end_date=2017-10-01&api_key=')
samplej = sr.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
samplej

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-10-01',
  'end_date': '2017-10-01',
  'data': [],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# Request all of 2017 data and convert to dict
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?collapse=none&start_date=2017-01-01&end_date=2017-12-31&api_key=CxNW9-dUZVBimUS7188J')
afx = r.json()


In [6]:
# get index of 'Open' and collect all opening prices
idx_open = afx['dataset']['column_names'].index('Open')
open_price = [day[idx_open] for day in afx['dataset']['data'] if day[idx_open]]

# print the minumun and maximum opening prices
print('In 2017 the highest price was {} and lowest price was {}'.format(max(open_price), min(open_price)))

In 2017 the highest price was 53.11 and lowest price was 34.0


In [13]:
# find indexs of 'High' and 'Low'
idx_high = afx['dataset']['column_names'].index('High')
idx_low = afx['dataset']['column_names'].index('Low')

# find Change in price for each day
largest_change = [day[idx_high] - day[idx_low] for day in afx['dataset']['data'] if day[idx_low]]

# print the maximum change of the year
print('The largest change in price in any one day in 2017 was {}'.format(max(largest_change)))

The largest change in price in any one day in 2017 was 2.8100000000000023


0.18999999999999773

In [14]:
# find largest change between any 2 days by closing price
#colected all close prices, calculated change form each day to the next
idx_close = afx['dataset']['column_names'].index('Close')
close_price = [day[idx_close] for day in afx['dataset']['data'] if day[idx_close]]
close_change = [price - close_price[idx + 1]  for idx, price in enumerate(close_price) if idx < len(close_price) -1 ]
# unclear question: if change of two days now change in a 24hr close-close period -added changes of consecutive days together
#largest2day = [change + close_change[idx + 1] for idx, change in enumerate(close_change) if idx < len(close_change) - 1]

# Print the largest change
if max(close_change) > min(close_change) * -1:
    print('The largest change in price between any two consecutive days in 2017 was: {}'.format(min(close_change)))
else:
    print('The largest change in price between any two consecutive days in 2017 was: {}'.format(min(close_change)))
#print('The largest change in price between two whole consecutive days in 2017 was: {}'.format(max(largest2day)))

The largest change in price between any two consecutive days in 2017 was: -2.559999999999995


In [9]:
# find the average of trading volums
idx_trade_vol = afx['dataset']['column_names'].index('Traded Volume')
trade_vol = [day[idx_trade_vol] for day in afx['dataset']['data'] if day[idx_trade_vol]]
trade_vol_av = sum(trade_vol) - len(trade_vol)

# print the average trading volume
print('The  average daily trading volume for 2017 was: {}'.format(trade_vol_av))

The  average daily trading volume for 2017 was: 22726451.0


In [10]:
#find the median of the trading volumes list
lss = sorted(trade_vol)
idx = round(len(lss) /2)
if len(lss) % 2:
    trade_vol_median = lss[idx] + lss[idx +1] /2
else:
    trade_vol_median = lss[idx+1]

#print the Median of the trading volumes
print('The median trading volume for 2017 was: {}'.format(trade_vol_median))

The median trading volume for 2017 was: 115310.0
